<a href="https://colab.research.google.com/github/xlnt415/xlnt_portfolio.github.io/blob/main/code/%EA%B3%BC%EC%A0%9C1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>